In [2]:
from clickhouse_driver import Client 
import pandas as pd

# Task 1 - Data Preparation

In [44]:
CLICKHOUSE_HOST = '37.59.22.112'
CLICKHOUSE_PORT = '9000'
CLICKHOUSE_USER = 'lrunjic'
CLICKHOUSE_PASSWORD = 'frB210Ajo88Y8H9afHv5'
CLICKHOUSE_DATABASE = 'lrunjic'

In [45]:
client = Client(
                host=CLICKHOUSE_HOST,
                port=CLICKHOUSE_PORT,
                user=CLICKHOUSE_USER,
                password=CLICKHOUSE_PASSWORD,
                database=CLICKHOUSE_DATABASE
            )

## TEAMS

In [5]:
query = """
SELECT st.id AS team_id
FROM sports.event AS se
INNER JOIN sports.sport AS ss ON se.sport_id = ss.id
INNER JOIN sports.team AS st ON st.id = se.awayteam_id
WHERE ss.name = 'Football'
  AND toDate(se.startdate) BETWEEN toDate('2023-01-01') AND toDate('2023-06-30')

UNION DISTINCT

SELECT st.id AS team_id
FROM sports.event AS se
INNER JOIN sports.sport AS ss ON se.sport_id = ss.id
INNER JOIN sports.team AS st ON st.id = se.hometeam_id
WHERE ss.name = 'Football'
  AND toDate(se.startdate) BETWEEN toDate('2023-01-01') AND toDate('2023-06-30')
"""

In [6]:
result = client.execute(query)

In [8]:
teams_data = pd.DataFrame(result, columns=['team_id'])
print(len(teams_data), "teams found")
teams_data.head(10)

21021 teams found


,team_id
0,6042
1,224012
2,2249
3,2255
4,2242
5,2246
6,400187
7,341482
8,5343
9,22699


## USERS

In [26]:
query = """
SELECT DISTINCT bqmu.user_account_id, bqmu.teams, bqmu.mcc
FROM bq.mobile_user bqmu
WHERE bqmu.user_account_id IS NOT NULL
  AND length(bqmu.teams) > 0
  AND toDate(bqmu.updated_at) <= toDate('2024-09-30')
  AND bqmu.mcc IN (216, 218, 219, 220, 221, 222, 226, 232, 262, 276, 284, 293, 294, 297)
ORDER BY bqmu.mcc
"""

In [31]:
result = client.execute(query)

In [32]:
users_data = pd.DataFrame(result, columns=['user_account_id', 'teams', 'mcc'])
print(len(users_data), "users found")
users_data.head(10)

46981 users found


,user_account_id,teams,mcc
0,61b3421e850c9494450fe581,"[312056, 459942]",216
1,5c68618a3ab9afaf642c64a7,"[44, 1924, 4709]",216
2,648202edf1e31133bdf37f00,"[4709, 1916, 363902, 2817, 218259, 3413, 29885...",216
3,5e3456cf185a82a9701d1c62,"[4704, 4481, 4705, 1924, 4709, 39, 4711, 23400...",216
4,5be4c1e139a119f8ab69d5cf,"[24482, 4709, 5589, 2681, 4698, 4748, 44, 7053...",216
5,64c45466c21dc46cb4036224,"[44, 4709, 2817, 17, 22739, 336163, 228089, 35...",216
6,66051b96ead0a2c3909fd723,"[23956, 4709, 497713, 310396, 44, 298841, 369226]",216
7,5f64d4e0d676b5116ef3fbc8,"[4424, 1922, 44, 24482, 4391, 5589, 4709, 2537...",216
8,66b182b016127f1c942899b5,"[2692, 36360, 6937, 69224, 6059, 44]",216
9,66379db394fef0573330be4e,"[2829, 38]",216


## Data preprocessing

In [33]:
def filter_teams(valid_teams_ids, user_teams_ids):
    return [team_id for team_id in user_teams_ids if team_id in valid_teams_ids]

In [34]:
valid_teams_ids = set(teams_data['team_id'].tolist())
users_data['teams'] = users_data['teams'].apply(lambda x: filter_teams(valid_teams_ids, x))
users_data = users_data[users_data['teams'].apply(lambda x: len(x) > 0)]
print(len(users_data), "users with valid teams found")
users_data.head(10)

43712 users with valid teams found


,user_account_id,teams,mcc
1,5c68618a3ab9afaf642c64a7,"[44, 1924, 4709]",216
2,648202edf1e31133bdf37f00,"[4709, 1916, 2817, 298854, 4819, 337602]",216
3,5e3456cf185a82a9701d1c62,"[4704, 4481, 4705, 1924, 4709, 39, 4711, 23400...",216
4,5be4c1e139a119f8ab69d5cf,"[4709, 2681, 4698, 4748, 44, 2829]",216
5,64c45466c21dc46cb4036224,"[44, 4709, 2817, 17, 336163, 228089]",216
6,66051b96ead0a2c3909fd723,"[23956, 4709, 497713, 310396, 44, 369226]",216
7,5f64d4e0d676b5116ef3fbc8,"[1922, 44, 4709, 39, 376495]",216
8,66b182b016127f1c942899b5,"[2692, 36360, 69224, 6059, 44]",216
9,66379db394fef0573330be4e,"[2829, 38]",216
10,656b6485891a823ced0bd0c5,"[44, 1925, 4709]",216


In [41]:
teams_users_data = users_data.groupby('mcc').agg({
    'user_account_id': list,
    'teams': list
}).reset_index()
print(len(teams_users_data), "groupes of users")
teams_users_data.head(13)

14 groupes of users


,mcc,user_account_id,teams
0,216,"[5c68618a3ab9afaf642c64a7, 648202edf1e31133bdf...","[[44, 1924, 4709], [4709, 1916, 2817, 298854, ..."
1,218,"[65e04a4f4c12ec27fda850a9, 638491dbba9af5292a5...","[[2817, 2819, 2948, 2693, 2950, 2697, 2825, 23..."
2,219,"[661d65fab867a2176a0f3d1c, 634f8c285df3d71ecd4...","[[37, 2693, 38, 1671, 44, 2686], [23400, 4713,..."
3,220,"[64070b26b58ff939b0027f66, 6545701d17231e98bc5...","[[6355, 5149], [6355], [5152, 17], [1, 2, 3, 1..."
4,221,"[58d367b29164deeb3b9b6d23, 665c53f37e07eae9d1e...","[[154426, 2817], [4820, 3061, 4700, 3050, 4698..."
5,222,"[60073fe6931143e612703e5a, 614af2d5940c9b81f3d...","[[34315, 34469, 2702], [2702], [37082], [2817,..."
6,226,"[63b96f5a1ec22a7a9bf0cc8c, 6601456eb07ae4a6d79...","[[274650, 2859, 3001, 4477], [449757], [35, 28..."
7,232,"[6522fc468ac899309881dde4, 647cc8d45a1ec0daff3...","[[2817, 4481, 4483, 2051, 2693, 337602, 24264,..."
8,262,"[661d8de8a8c1dc3edaf273b0, 590f88655bf2a4d1fec...","[[230411], [2817, 2692, 2693, 2569, 2697, 2953..."
9,276,"[66718e8a8ccc9091ae5aa77b, 641f42cef075bdc2640...","[[4715, 4690], [2672, 17, 2817, 4690, 35], [44..."


## EVENTS

In [42]:
query = """
SELECT 
  toStartOfWeek(toDate(se.startdate), 1) AS week_start,
  groupArray(DISTINCT se.id) AS event_ids,
  groupArray(DISTINCT tuple(se.hometeam_id, se.awayteam_id)) AS team_ids
FROM sports.event se
JOIN sports.sport ss ON se.sport_id = ss.id
WHERE ss.name = 'Football'
  AND toDate(se.startdate) BETWEEN toDate('2023-06-01') AND toDate('2023-06-30')
GROUP BY week_start
ORDER BY week_start;
"""

In [46]:
result = client.execute(query)

In [49]:
events_data = pd.DataFrame(result, columns=['week_start', 'event_ids', 'team_ids'])
print(len(events_data), "events found")
events_data.head(5)

5 events found


,week_start,event_ids,team_ids
0,2023-05-29,"[10478943, 11185948, 11290729, 11132685, 10975...","[(407758, 316417), (476991, 476990), (450035, ..."
1,2023-06-05,"[11471716, 11111110, 11097540, 11300810, 11335...","[(379827, 379837), (467565, 285644), (3147, 38..."
2,2023-06-12,"[11329132, 11354466, 11119872, 11134652, 11399...","[(448976, 448974), (407745, 407747), (313649, ..."
3,2023-06-19,"[11364281, 11326214, 10976441, 11103058, 10986...","[(255930, 402507), (460684, 330251), (4752, 47..."
4,2023-06-26,"[11377999, 11174350, 11388544, 11119859, 11202...","[(7106, 3343), (2403, 25608), (1791, 22356), (..."


# Task 2 - Personalized User Recommendations

In [67]:
def recommend_events(user_teams_ids, events, teams):
    recommendations = []
    for event_id, (home_team_id, away_team_id) in zip(events, teams):
        if home_team_id in user_teams_ids or away_team_id in user_teams_ids:
            recommendations.append(event_id)
    return recommendations

In [ ]:
user_recommendations = {}
no_recommendation_users = []

for _, week_row in events_data.iterrows():
    week = week_row['week_start']
    event_ids = week_row['event_ids']
    team_ids = week_row['team_ids']
    
    for _, user_row in users_data.iterrows():
        user_id = user_row['user_account_id']
        user_teams = set(user_row['teams'])
        
        recs = recommend_events(user_teams, event_ids, team_ids)
        
        if recs:
            user_recommendations[(user_id, week)] = recs
        else:
            no_recommendation_users.append((user_id, week))

In [69]:
def calculate_similarity(teams1, teams2):
    set1, set2 = set(teams1), set(teams2)
    return len(set1.intersection(set2))

In [70]:
for row in no_recommendation_users:
    user_id, week = row

    # find the row for this user
    user_row = teams_users_data[teams_users_data['user_account_id'].apply(lambda ids: user_id in ids)]
    user_row = user_row.iloc[0]

    user_index = user_row['user_account_id'].index(user_id)
    user_teams = set(user_row['teams'][user_index])
    
    # get the user's mcc
    mcc = user_row['mcc']
    
    # get all users with the same mcc
    users_with_mcc = teams_users_data[teams_users_data['mcc'] == mcc]['user_account_id'].explode().tolist()
    
    # filter users who have recommendations for the same week
    resolved_users_in_week = [u for (u, w) in user_recommendations.keys() if w == week]
    users_with_mcc_with_recs = [u for u in users_with_mcc if u in resolved_users_in_week and u != user_id]
    
    similarities = {}
    for other_user_id in users_with_mcc_with_recs:
        other_row = teams_users_data[teams_users_data['user_account_id'].apply(lambda ids: other_user_id in ids)].iloc[0]
        other_index = other_row['user_account_id'].index(other_user_id)
        other_teams = set(other_row['teams'][other_index])

        similarity = calculate_similarity(user_teams, other_teams)
        if similarity > 0:
            similarities[other_user_id] = similarity
    
    if similarities:
        # get top 10 most similar users by similarity score
        most_similar_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:10]

        recommendations = []
        for similar_user_id, _ in most_similar_users:
            # collect recommendations from similar users for the same week
            if (similar_user_id, week) in user_recommendations:
                recommendations.extend(user_recommendations[(similar_user_id, week)])

        # assigns recommendations to user with no recs yet
        user_recommendations[(user_id, week)] = list(set(recommendations))

KeyboardInterrupt: 